what do i need to do?

I need to:
    * make a basic gan out of biodeg
    * make a basic gan out of druglike
    * figure out the predictors that are being used for biodegradability
        * try out those descriptors for predictive power on the druglike set 
        * (also, do I want to make a bigger druglike set?)

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf


import matplotlib.pyplot as plt

import seaborn as sns
from math import floor, ceil
from pylab import rcParams
from sklearn.utils import shuffle
from sklearn.feature_selection import VarianceThreshold 

%matplotlib inline

In [2]:
def encode(series): 
    return pd.get_dummies(series.astype(str))

In [3]:
train_x = pd.read_csv('../../big datasets/drugml/x_train_res.csv')
train_y = pd.read_csv('../../big datasets/drugml/y_train_res.csv')
test_x = pd.read_csv('../../big datasets/drugml/x_test.csv')
test_y = pd.read_csv('../../big datasets/drugml/y_test.csv')

In [4]:
from sklearn.preprocessing import StandardScaler
xscaler = StandardScaler().fit(train_x)
train_x = xscaler.transform(train_x)
testscaler = StandardScaler().fit(test_x)
test_x = testscaler.transform(test_x)


C:\Users\ellie\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ellie\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [5]:
x_train = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
x_test = pd.DataFrame(test_x)
test_y = pd.DataFrame(test_y)

In [6]:
#x_train = x_train.drop(['Unnamed: 0'], axis=1)

train_y = train_y.drop(['Unnamed: 0'], axis=1)
y_train = encode(train_y)
#y_train = encode(train_y)
#x_test = test_x.drop(['Unnamed: 0'], axis=1)
test_y = test_y.drop(['Unnamed: 0'], axis=1)
y_test = encode(test_y)
#y_test = encode(test_y)

In [8]:
x_train.shape

(1626, 280)

In [110]:
random_state = 42
np.random.seed(random_state)
tf.set_random_seed(random_state)

In [111]:
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    
    return out_layer



In [94]:
param_dist = {'hidden' : [10, 20, 30, 40, 50],
             'prob' : [0.2, 0.4, 0.6, 0.8],
             'size': [32, 45, 60],
             'rate' : [0.05, 0.01, 0.001, 0.001]}


In [95]:
y_test = pd.DataFrame(y_test)
y_test.head()

,Class_0,Class_1
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [128]:
 #### use meeeeee !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
n_hidden_1 = 40
n_input = train_x.shape[1]
n_classes = y_train.shape[1]

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
    }

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
    }

keep_prob = tf.placeholder("float")

training_epochs = 3
display_step = 1
batch_size = 60

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
    
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.35
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(predictions, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
    
    
    confusion = tf.confusion_matrix(labels=tf.argmax(y, 1), predictions=tf.argmax(predictions, 1), num_classes=2) 
    print(confusion.eval({x: x_test, y: y_test, keep_prob: 1.0}))    

Epoch: 0001 cost= 13.583344702
Epoch: 0002 cost= 5.395451939
Epoch: 0003 cost= 2.228134196
Optimization Finished!
Accuracy: 0.800813
[[ 42   0]
 [ 49 155]]


In [71]:
from sklearn.utils import shuffle

In [72]:
#### use meeeeee 
#### use meeeeee
n_hidden_1 = 60
n_input = train_x.shape[1]
n_classes = train_y.shape[1]

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
    }

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
    }

keep_prob = tf.placeholder("float")

training_epochs = 10
display_step = 1
batch_size = 60

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
    
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_train, y_train = shuffle(x_train, y_train, random_state=12)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.35
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    x_test, y_test = shuffle(x_test, y_test, random_state=12)
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(predictions, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
    
    
    confusion = tf.confusion_matrix(labels=tf.argmax(y, 1), predictions=tf.argmax(predictions, 1), num_classes=2) 
    print(confusion.eval({x: x_test, y: y_test, keep_prob: 1.0}))    

Epoch: 0001 cost= 0.000000000
Epoch: 0002 cost= 0.000000000
Epoch: 0003 cost= 0.000000000
Epoch: 0004 cost= 0.000000000
Epoch: 0005 cost= 0.000000000
Epoch: 0006 cost= 0.000000000
Epoch: 0007 cost= 0.000000000
Epoch: 0008 cost= 0.000000000
Epoch: 0009 cost= 0.000000000
Epoch: 0010 cost= 0.000000000
Optimization Finished!
Accuracy: 1.0
[[246   0]
 [  0   0]]


In [ ]:
sa

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
estimator = run_model(setup_model)

In [97]:
param_dist = {'hidden' : [10, 20, 30, 40, 50],
             'prob' : [0.2, 0.4, 0.6, 0.8],
             'size': [32, 45, 60],
             'rate' : [0.05, 0.01, 0.001, 0.001]}

In [99]:
hid = [30, 35, 40, 45, 50, 55, 60, 65, 70]
pro = [0.3, 0.35, 0.4]
rat = [0.1]

In [124]:
hid = [30, 40, 50]
pro = [0.3, 0.5, 0.7]
rat = [0.1]
siz = [60]

In [125]:
acc = []
for hidden in hid:
    for prob in pro:
        for rate in rat:
                
            mod = []
                    
            n_hidden_1 = hidden
            n_input = train_x.shape[1]
            n_classes = y_train.shape[1]
            weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
    }

            biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
    }

            keep_prob = tf.placeholder("float", None)

            training_epochs = 20
            display_step = 2
            batch_size = 60

            x = tf.placeholder("float", [None, n_input])
            y = tf.placeholder("float", [None, n_classes])
            predictions = multilayer_perceptron(x, weights, biases, keep_prob)
    
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
            optimizer = tf.train.AdamOptimizer(learning_rate=rate).minimize(cost)
            
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())

                for epoch in range(training_epochs):
                    avg_cost = 0.0
                    total_batch = int(len(x_train) / batch_size)
                    x_train, y_train = shuffle(x_train, y_train, random_state=12)
                    x_batches = np.array_split(x_train, total_batch)
                    y_batches = np.array_split(y_train, total_batch)
                for i in range(total_batch):
                    batch_x, batch_y = x_batches[i], y_batches[i]
                    _, c = sess.run([optimizer, cost], 
                                feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: prob})
                    avg_cost += c / total_batch
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost=", \
                    "{:.9f}".format(avg_cost))
                correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(predictions, 1))
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                mod.append(accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
                
    
                confusion = tf.confusion_matrix(labels=tf.argmax(y, 1), predictions=tf.argmax(predictions, 1), num_classes=2) 
                mod.append(confusion.eval({x: x_test, y: y_test, keep_prob: 1.0}))
                mod.append(hidden)
                mod.append(prob)
                mod.append(rate)
                
                acc.append(mod)


In [126]:
sorted(acc, key=lambda x: x[0])

[[0.68292683, array([[ 42,   0],
         [ 78, 126]]), 30, 0.7, 0.1], [0.7479675, array([[ 42,   0],
         [ 62, 142]]), 40, 0.5, 0.1], [0.7479675, array([[ 42,   0],
         [ 62, 142]]), 50, 0.3, 0.1], [0.75609756, array([[ 41,   1],
         [ 59, 145]]), 40, 0.7, 0.1], [0.7886179, array([[ 42,   0],
         [ 52, 152]]), 50, 0.5, 0.1], [0.79268295, array([[ 42,   0],
         [ 51, 153]]), 30, 0.5, 0.1], [0.796748, array([[ 41,   1],
         [ 49, 155]]), 30, 0.3, 0.1], [0.796748, array([[ 41,   1],
         [ 49, 155]]), 40, 0.3, 0.1], [0.8089431, array([[ 42,   0],
         [ 47, 157]]), 50, 0.7, 0.1]]

In [ ]:
[0.800813, array([[ 41,   1], [ 48, 156]]), 30, 20, 0.1], 
[0.800813, array([[ 42,   0], [ 49, 155]]), 70, 40, 0.1], 
[0.8130081, array([[ 42,   0], [ 46, 158]]), 70, 60, 0.1], 
[0.81707317, array([[ 33,   9], [ 36, 168]]), 70, 40, 0.001], 
[0.8292683, array([[ 41,   1], [ 41, 163]]), 50, 20, 0.1]]

In [ ]:
[0.7195122, array([[ 42,   0], [ 69, 135]]), 30, 60, 0.1], 
[0.7357724, array([[ 42,   0], [ 65, 139]]), 50, 45, 0.1], 
[0.7479675, array([[ 42,   0], [ 62, 142]]), 70, 32, 0.1], 
[0.7479675, array([[ 20,  22], [ 40, 164]]), 100, 32, 0.001], 
[0.7723577, array([[ 42,   0], [ 56, 148]]), 50, 32, 0.1], 
[0.7723577, array([[ 42,   0], [ 56, 148]]), 100, 32, 0.1], 
[0.7804878, array([[ 42,   0], [ 54, 150]]), 30, 45, 0.1], 
[0.7804878, array([[ 17,  25], [ 29, 175]]), 50, 60, 0.001], 
[0.7886179, array([[ 42,   0], [ 52, 152]]), 50, 60, 0.1], 
[0.7886179, array([[ 42,   0], [ 52, 152]]), 100, 45, 0.1], 
[0.7886179, array([[ 29,  13], [ 39, 165]]), 100, 45, 0.001], 
[0.7886179, array([[ 42,   0], [ 52, 152]]), 100, 60, 0.1], 
[0.796748, array([[ 41,   1], [ 49, 155]]), 30, 32, 0.1], 
[0.796748, array([[ 18,  24], [ 26, 178]]), 70, 45, 0.001], 
[0.8130081, array([[ 41,   1], [ 45, 159]]), 70, 45, 0.1], 
[0.8211382, array([[ 42,   0], [ 44, 160]]), 70, 60, 0.1], # explore same batch sizes, explore hidden less than 100
[0.85365856, array([[ 32,  10], [ 26, 178]]), 70, 32, 0.001]] # explore 0.1 versus 0.01 versus 0.001

In [ ]:
[0.7706422, array([[110,  36],
         [ 14,  58]]), 10, 0.7, 60, 0.1], 
[0.77522933, array([[117,  29],
         [ 20,  52]]), 30, 0.5, 60, 0.1], 
[0.77522933, array([[127,  19],
         [ 30,  42]]), 70, 0.7, 60, 0.1], 
[0.7844037, array([[131,  15],
         [ 32,  40]]), 50, 0.3, 60, 0.1], 
[0.7844037, array([[133,  13],
         [ 34,  38]]), 50, 0.7, 60, 0.1], # best 0.1 learning rate, dropout 0.5 or 0.3, best hidden unknown...
[0.78899086, array([[129,  17],
         [ 29,  43]]), 30, 0.7, 60, 0.1], 
[0.79816514, array([[113,  33],
         [ 11,  61]]), 10, 0.5, 60, 0.1], 
[0.79816514, array([[138,   8],
         [ 36,  36]]), 50, 0.5, 60, 0.1], 
[0.8027523, array([[119,  27],
         [ 16,  56]]), 30, 0.3, 60, 0.1], 
[0.8027523, array([[137,   9],
         [ 34,  38]]), 70, 0.3, 60, 0.1], 
[0.8348624, array([[128,  18],
         [ 18,  54]]), 70, 0.5, 60, 0.1]

In [ ]:
 [0.83027524, array([[132,  14],
         [ 23,  49]]), 50, 0.3, 60, 0.1]] # narrow in on 50 and 0.3

In [ ]:
[0.8211009, array([[126,  20],
         [ 19,  53]]), 60, 0.35, 60, 0.1], winner 
hyperparameters: 60, .35, 60, 0.1

In [ ]:
from random import randint
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Load the dataset
X, Y = x_train, y_train

# Create model for KerasClassifier
def create_model(hparams1,
                 hparams2,
                 hparams3,
                 hparams4):
    
    n_hidden_1 = hparams1
    n_input = train_x.shape[1]
    n_classes = train_y.shape[1]

    weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
    }

    biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
    }

    keep_prob = tf.placeholder("float")

    training_epochs = 200
    display_step = 90
    batch_size = hparams3

    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])
    predictions = multilayer_perceptron(x, weights, biases, keep_prob)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=hparams4).minimize(cost)

    return

with tf.Session() as sess:
    model = sess.run(tf.global_variables_initializer())

    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                        feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: hparams2
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
            "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(predictions, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
    
    
    confusion = tf.confusion_matrix(labels=tf.argmax(y, 1), predictions=tf.argmax(predictions, 1), num_classes=2) 
    print(confusion.eval({x: x_test, y: y_test, keep_prob: 1.0}))

# Specify parameters and distributions to sample from
hparams1 = randint(10, 100)
hparams2 = randint(0,10)*0.1
hparams3 = randint(30,60)
hparams4 = [0.1, 0.05, 0.01, 0.001]

# Prepare the Dict for the Search
param_dist = dict(hparams1=hparams1, 
                  hparams2=hparams2, 
                  hparams3=hparams3, 
                  hparams4=hparams4)

# Search in action!
n_iter_search = 16 # Number of parameter settings that are sampled.
random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(X, Y)

# Show the results
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Load the dataset

# Create model for KerasClassifier
   
n_hidden_1 = hparams1
n_input = train_x.shape[1]
n_classes = train_y.shape[1]

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
    }

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
    }

keep_prob = tf.placeholder("float")

training_epochs = 200
display_step = 90
batch_size = hparams3

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
    
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=hparams4).minimize(cost)

with tf.Session() as sess:
    model = sess.run(tf.global_variables_initializer())

    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                        feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: hparams2
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
            "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(predictions, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
    
    
    confusion = tf.confusion_matrix(labels=tf.argmax(y, 1), predictions=tf.argmax(predictions, 1), num_classes=2) 
    print(confusion.eval({x: x_test, y: y_test, keep_prob: 1.0}))

# Specify parameters and distributions to sample from
hparams1 = randint(10, 100)
hparams2 = randint(0,10)*0.1
hparams3 = randint(30,60)
hparams4 = [0.1, 0.05, 0.01, 0.001]

# Prepare the Dict for the Search
param_dist = dict(hparams1=hparams1, 
                  hparams2=hparams2, 
                  hparams3=hparams3, 
                  hparams4=hparams4)

# Search in action!
n_iter_search = 16 # Number of parameter settings that are sampled.
random_search = RandomizedSearchCV(estimator=model, 
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search)
random_search.fit(X, Y)

# Show the results
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
def model(X_train, Y_train, X_val, Y_val):
    
    model = Sequential()
    model.add(Dense({{choice([10, 20, 40, 104])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([10, 20, 40, 104])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    
    if conditional({{choice(['two', 'three'])}}) == 'three':
        model.add(Dense({{choice([10, 20, 40, 104])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))
        
    model.add(Dense(10))
    model.add(Activation('softmax'))
    adam = keras.optimizers.Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    rmsprop = keras.optimizers.RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    sgd = keras.optimizers.SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
   
    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = adam
    elif choiceval == 'rmsprop':
        optim = rmsprop
    else:
        optim = sgd
        
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=optim)
    model.fit(X_train, Y_train,
              batch_size={{choice([128,256,512])}},
              nb_epoch=20,
              verbose=2,
              validation_data=(X_val, Y_val))
    score, acc = model.evaluate(X_val, Y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [ ]:
X_train, Y_train, X_val, Y_val = x_train, y_train, x_test, y_test

In [ ]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import keras

In [ ]:
data = X_train, Y_train, X_val, Y_val
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                      notebook_name='Neural Network-Copy1')

need to make this different... this needs to have a different train/test layout because the test data is not getting called properly. try https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428 this method here

_________________________________

  correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(predictions, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))


    confusion = tf.confusion_matrix(labels=tf.argmax(y, 1), predictions=tf.argmax(predictions, 1), num_classes=2) 
    print(confusion.eval({x: x_test, y: y_test, keep_prob: 1.0}))

 total_error = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
    unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y, predictions)))
    R_squared = tf.subtract(1.0, tf.divide(unexplained_error, total_error))
    print(R_squared.eval({x: x_test, y: y_test, keep_prob: 1.0}))

sklearn naive random oversampling (imbalanced data)
https://imbalanced-learn.readthedocs.io/en/stable/over_sampling.html
https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.tensorflow.balanced_batch_generator.html

* look for network architecture from paper that used the QSAR 
* lasso 
* fix for sparse data
* find any columns that are uniform (or very low variation)
* normalize 
* tensorboard
* early stopping - ask rainie if i need help
* put layer in after dropout
* if oversampling, up the dropout (is there a ratio)
* test set needs to be balanced but not oversampled 
* use their train/test split, then shuffle the data
* combine the set up cells and do a for loop for the number of nodes (like 5-50 at 5 or 10 node increments)
    * use on waffle?
    


create environment (homoganize)
install everything i need into it
* do the wget thing

    MSE = tf.metrics.mean_squared_error(tf.cast(y_test, tf.float32),
    predictions,
    weights=None,
    metrics_collections=None,
    updates_collections=None,
    name=None)
    print("MSE:", MSE)
    fn = tf.metrics.false_negatives(
    tf.cast(y_test, tf.float32),
    predictions,
    weights=None,
    metrics_collections=None,
    updates_collections=None,
    name=None)
    tn = tf.metrics.true_negatives(
    tf.cast(y_test, tf.float32),
    predictions,
    weights=None,
    metrics_collections=None,
    updates_collections=None,
    name=None)
    fp = tf.metrics.false_positives(
    tf.cast(y_test, tf.float32),
    predictions,
    weights=None,
    metrics_collections=None,
    updates_collections=None,
    name=None)
    tp = tf.metrics.true_positives(
    tf.cast(y_test, tf.float32),
    predictions,
    weights=None,
    metrics_collections=None,
    updates_collections=None,
    name=None)
    print("FN:", fn, "TN:", tn, "FP:", fp, "TP:", tp)
    total_error = tf.reduce_sum(tf.square(tf.subtract(tf.cast(y_test, tf.float32), tf.reduce_mean(tf.cast(y_test, tf.float32)))))
    unexplained_error = tf.reduce_sum(tf.square(tf.subtract(tf.cast(y_test, tf.float32), tf.cast(predictions, tf.float32))))
    R_squared = tf.subtract(1.0, tf.divide(unexplained_error, total_error))
    print(R_squared)

df = pd.DataFrame({'num_legs': [2, 4, 8, 0],
...                    'num_wings': [2, 0, 0, 0],
...                    'num_specimen_seen': [10, 2, 1, 8]},
...                   index=['falcon', 'dog', 'spider', 'fish'])

df

df1 = df.iloc[:,1:3]
df1.head()